In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [53]:
print(pd.__version__)

2.1.4


In [2]:
data=pd.read_csv("../input/amazon-fine-food-reviews/Reviews.csv")

In [3]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Data Preprocessing

In [4]:
# Drop duplicate
data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},inplace=True)#dropping duplicates
data.dropna(axis=0,inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 393914 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      393914 non-null  int64 
 1   ProductId               393914 non-null  object
 2   UserId                  393914 non-null  object
 3   ProfileName             393914 non-null  object
 4   HelpfulnessNumerator    393914 non-null  int64 
 5   HelpfulnessDenominator  393914 non-null  int64 
 6   Score                   393914 non-null  int64 
 7   Time                    393914 non-null  int64 
 8   Summary                 393914 non-null  object
 9   Text                    393914 non-null  object
dtypes: int64(5), object(5)
memory usage: 33.1+ MB


In [6]:
example = {}
example['Id'] = list(data.loc[data['Id'] == 13]['Id'])
example['Summary'] = list(data.loc[data['Id'] == 13]['Summary'])
example['Text'] = list(data.loc[data['Id'] == 13]['Text'])
print(example)

{'Id': [13], 'Summary': ['My Cats Are Not Fans of the New Food'], 'Text': ["My cats have been happily eating Felidae Platinum for more than two years. I just got a new bag and the shape of the food is different. They tried the new food when I first put it in their bowls and now the bowls sit full and the kitties will not touch the food. I've noticed similar reviews related to formula changes in the past. Unfortunately, I now need to find a new food that my cats will eat."]}


In [7]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer

!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
def get_wordnet_pos(treebank_tag):
    """
    return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

In [70]:

example = "<br>I ordered a pizza yesterday from @Dominos. I have to say that it was the greatest pizza I have eaten in recent times!! It was filled with cheese and delicious toppings. #CheeseIsLife</br>"
print("Example: ", example)

# Initialize resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    """
    Preprocesses text by performing tokenization, filtering, and lemmatization.
    """

    text = text.lower()
    
    #Remove HTML Tags
    text = re.sub(r"<.*?>", '', text)
    
    #Remove URL
    text = re.sub(r"https?://\S+|www\.\S+", '', text)
    
    #Remove mentions and hashtags
    text = re.sub(r"@\w+|#\w+", '', text)
    
    text = re.sub(r"[{}]+".format(re.escape(string.punctuation)), ' ', text)
    
    
    # Step 1: Tokenization
    tokens = word_tokenize(text)
    # print("Tokens: ", tokens)

    # Step 2: Remove unnecessary tokens
    cleaned_tokens = []
    for token in tokens:
        if token.isdigit():  # Remove numeric text
            continue
        if token in stop_words or token in string.punctuation:  # Remove stop words and punctuation
            continue
        cleaned_tokens.append(token)
    # print("Cleaned Tokens: ", cleaned_tokens)

    #POS Tagged
    tagged_tokens = pos_tag(cleaned_tokens)
    # print("Tagged Tokens: ", tagged_tokens)
    
    # Lemmatization and Noun Filtration
    lemmatized_tokens = []
    # noun_unigram = []
    for token, tag in tagged_tokens:
        lemma_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(token, lemma_tag)
        lemmatized_tokens.append(lemma) 
      
    
    # print("Lemmatized Tokens: ", lemmatized_tokens)
        
    return lemmatized_tokens
    

# Example input
processed_example = preprocess_text(example)
print(processed_example)

Example:  <br>I ordered a pizza yesterday from @Dominos. I have to say that it was the greatest pizza I have eaten in recent times!! It was filled with cheese and delicious toppings. #CheeseIsLife</br>
['order', 'pizza', 'yesterday', 'say', 'great', 'pizza', 'eaten', 'recent', 'time', 'fill', 'cheese', 'delicious', 'topping']


In [75]:
def get_noun_unigrams(preprocessed_tokens):
    #POS Tagged
    tagged_tokens = pos_tag(preprocessed_tokens)


    noun_unigram = []
    for token, tag in tagged_tokens:
        if(tag[:2] == 'NN'):
            noun_unigram.append(token)

    return noun_unigram

In [88]:

# Shuffle the dataset
data_shuffled = data.sample(frac=1, random_state=42)  # Use random_state for reproducibility

# Select 1,000 random reviews
data_subset = data_shuffled.head(5000)
data_subset.head()


data_preprocessed = {}
data_preprocessed['Id'] = data_subset['Id']
data_preprocessed['Text'] = data_subset['Text']
data_preprocessed['Preprocessed_Text'] = data_subset['Text'].apply(preprocess_text)
data_preprocessed['Noun_Unigram'] = data_preprocessed['Preprocessed_Text'].apply(get_noun_unigrams)

data_preprocessed = pd.DataFrame(data_preprocessed)
data_preprocessed.head()

,Id,Text,Preprocessed_Text,Noun_Unigram
256467,256468,I received a small jar of Bali's Best Coffee f...,"[receive, small, jar, bali, best, coffee, chri...","[jar, bali, coffee, christmas, store, amazon, ..."
4073,4074,"I will admit, the Icicle does allow you to plu...","[admit, icicle, allow, plug, mic, xlr, jack, c...","[admit, icicle, plug, xlr, jack, computer, use..."
64193,64194,"This house had a number of broken pieces, but ...","[house, number, broken, piece, repairable, ice...","[house, number, piece, ice, candy, house]"
160944,160945,My wife loves using this mix for her crockpot ...,"[wife, love, use, mix, crockpot, meal, try, ot...","[wife, love, use, mix, crockpot, meal, try, ot..."
103614,103615,This reminds me of Fentimin's Curiosity Cola w...,"[reminds, fentimin, curiosity, cola, botanical...","[reminds, curiosity, cola, undertone, coke, di..."


# Opinion Feature Extraction


## Context Similarity

In [77]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 32.0 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [78]:
import spacy 

nlp = spacy.load('en_core_web_md') 

print("Enter word") 
word = input() 
reference_word = "food"

noun_token = nlp(word) 
reference_token = nlp(reference_word)
print(noun_token.text, noun_token.has_vector, noun_token.vector_norm) 
print("Similarity:", reference_token.similarity(noun_token)) 


Enter word


 cat


cat True 63.18849331586163
Similarity: 0.2232031355893895


In [81]:
import spacy

nlp = spacy.load('en_core_web_md')

def calculate_cosine_similarity(tokens, reference_word = 'food', delta = 0.4):
    reference_token = nlp(reference_word)
    # print(reference_token.text, reference_token.has_vector, reference_token.vector_norm) 
    
    similar_tokens = []
    for noun in tokens:
        noun_token = nlp(noun)
        # print(noun_token.text, noun_token.has_vector, noun_token.vector_norm) 
        
        if (noun_token.has_vector):
            similarity = reference_token.similarity(noun_token)
            # print(f"{noun_token} : {similarity:.4f}")
            if similarity > delta:
                similar_tokens.append((noun, similarity))

    return similar_tokens



In [89]:
data_similar = {}
data_similar['Id'] = data_preprocessed['Id']
data_similar['Similar_Nouns'] = data_preprocessed['Noun_Unigram'].apply(calculate_cosine_similarity)

data_similar = pd.DataFrame(data_similar)
data_similar.head()

,Id,Similar_Nouns
256467,256468,"[(coffee, 0.5149293216837122)]"
4073,4074,[]
64193,64194,[]
160944,160945,"[(meal, 0.5870303326567043), (grocery, 0.55804..."
103614,103615,[]


In [91]:
data_preprocessed.to_csv('preprocessed_text.csv', index = False)
data_similar.to_csv('similar_nouns.csv',index = False)

In [83]:
similar_score_example = calculate_cosine_similarity(processed_example)
print(similar_score_example)

[('pizza', 0.46639817930773697), ('pizza', 0.46639817930773697), ('cheese', 0.41129490117464185), ('delicious', 0.5739990305090508)]
